本节主要介绍attention机制，前部分都在闲聊，有兴趣的可以看看视频。<br>
中休讲的是google's Multilingual NMT system

### Attention
就像人为翻译的时候，当句子比较长的时候，我们可能不能一次性对整个句子进行翻译，而是通过前后比对，逐步进行翻译。attention机制，就是这么一种前后比对的机制，它使得机器可以在翻译当前的单词的时候，获得encoder隐藏层对应单词周围（local还是global取决于信息获取方式）的h值，使得机器翻译的时候，能够更加关注对应单词的原文。<br>
在旧版本的MT中，decoder的信息，仅来自与encoder的最后输出，也就是h_t（他包含了原句的所有信息）。为了实现attention机制，我们的想法是，我们不但只使用encoder隐藏层最后的输出，而是利用其各个时段输出的内容。<br>
确定信息源后，下一个应该关心的是，如何配对，即找到对应的原单词。这就回到了alignment的老问题了。我们这里使用的方法是：compare target and source hidden states，为encoder的每个状态计算一个score（$score(h_{t-1}, \hat{h}_s)$），然后用softmax计算各个状态的权重（全局，local的话使用window）。最后集中这些状态，得到我们想要的context vector:
<center>$c_t\ =\ \sum_sa_t(s)\hat{h}_s$</center>
我们是用$c_t$来辅助预测单词。<br>
那么我们应该如何计算这个score 呢？？
<center>$score(h_t, \hat{h}_s)\ =\ h_t^T\hat{h}_s$</center>
or
<center>$score(h_t, \hat{h}_s)\ =\ h_t^TW_a\hat{h}_s$(Bilinear form)</center>
第二个更优，因为that matrix can effectively learn how much weight to put on different part of the dot product, to sort of have an idea of where to pay attention.其实不是很了解，这里的attention是指什么，因为score本身不就是用来代表attention的吗？<br>
What you would like to do is kind of have interaction terms that look at $h_t$ and $h_s$ together. And even the dot product kind of has this interaction between $h_t$ and $h_s$. and this is a more sophisticated way of getting an interaction between $h_t$ and $h_s$<br>
Whereas if you're using this model with only a single layer of neural network, you don't actually get interactions between $h_t$ and $h_s$. because you've got the sort of two parts of this vector and each of them is multiplied by a separate part of this matrix. and then you put it through a tanh, but that just rescales it element-wise. and then you multiply it by a vector, but that just rescales it element-wise. So there's no place that $h_t$ and $h_s$ acutally interact with each other.其原理就和单层神经元，不能解决xor问题一样（you can't get the two things to interact with each other）。$score=v_a^Ttanh(W_a[h_t;\hat{h}_s])$

一般四五个单词长的句子的翻译，反而比长句子的更差，他可能的原因是很多短句，都被误认为是短语而导致的。

### coverage
为了确保能够关注到句子的不同的部位，而引入的机制。
#### doubly attention
sort of working out attention in both dimensions:
<center>$c_t\ =\ -log(P(y|x))+\lambda\sum_i^L(1-\sum_t^C\alpha_{ti})^2$</center>

![douby attention](https://raw.githubusercontent.com/HuangYiran/Natural_Language_Processing_with_Deep_Learning/master/douby_attention.png)

#### Extending attention with linguistic ideas previously in alignment models
- NMT model with coverage-based attention
- More substantive medels of attention using: position(IBM2) + Markov(HMM) + fertility(IBM3-5) + alignment symmetry(BerkeleyAligner)<br>
挺有意思的，有兴趣的时候再看吧。

### Sequence Model Decoders:
#### Exhaustive search
- score each and every possible translation, pick the best one 开玩笑的


#### Ancestral Sampling
- one symbol at a time from $\hat{x}_t \backsim x_t|x_{t-1},...,x_1, Y$
- Until $\hat{x}_t = <eos>
- Repeat


#### Greedy Search
- pick the most likely symbol each time:
<center>$\hat{x}_t\ =\ argmax_x log p(x|x_{<t},Y)$</center>
- until $\hat{x}_t = <eos>$

Pros:Super-efficient,both computation and memory<br>
cons: heavily suboptimal

#### beam search
把greedy search中的每步取最优，改为每步取最优的k个。其中k为beam的大小。因此增长为以k为基的指数。